Bye


In [2]:
#import imutils
import time
import cv2
import numpy as np
import queue 

previousFrame = None
count = 0
test = 0
temp_frame = None
rect = None
newThresh = None
#return only  the index of the largest contour
def getLargestCountour(contours):
    largestArea = 0
    index = 0
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area>largestArea):
                largestArea=area
                index= i
    if cv2.contourArea(contours[index]) < 5000:
        index =-1
    print(largestArea)            
    return index       

def searchForMovement(contours, frame):
    #get the index of the largest contour
    index = getLargestCountour(contours)
    
    if index >=0 and index <= len(contours) -1:
        cnt = contours[index]
        M = cv2.moments(cnt) #get some properties from the biggest countour
        cx = int(M['m10']/M['m00']) #x axis centroid
        cy = int(M['m01']/M['m00']) #y axis centroid
        cv2.circle(frame,(cx,cy),5,[255,0,0],-1) #draw the center of the countour mass
        
        hull = cv2.convexHull(cnt,returnPoints = False)
        defects = cv2.convexityDefects(cnt,hull)
        #hand = hsv[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]
        #https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html#contour-features
        if defects is not None:
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                cv2.line(frame,start,end,[0,255,0],2)
                rows,cols = frame.shape[:2]
                [vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2,0,0.01,0.01)
                lefty = int((-x*vy/vx) + y)
                righty = int(((cols-x)*vy/vx)+y)
                cv2.line(frame,(cols-1,righty),(0,lefty),(0,0,255),2)

            return frame

def trackMotion(frame,prev, sensitivity_value):

        # Convert to grayscale and blur it for better frame difference
        # frame = cv2.bilateralFilter(frame, 7, 150, 150)
        frame = cv2.blur(frame, (9,9)) # blur the original frame "very important for the final image"
       #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        previousFrame = prev
        global newThresh

        
        kernelErosion = np.ones((1,1),np.uint8) #kernel for erosion
        kernelDilation = np.ones((8,8),np.uint8) #kernel for dilation
        if previousFrame is None:
            previousFrame = frame.copy()
            prevHist =0
            #print('hello')
            return frame, previousFrame, frame, frame   

        frameDiff = cv2.absdiff(previousFrame, frame)
        #frameDiff = cv2.cvtColor(frameDiff, cv2.COLOR_BGR2GRAY)
        
        thresh = cv2.threshold(frameDiff, sensitivity_value, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
      
        hist = cv2.calcHist([thresh], [0], None, [256], [0,256])

        if hist[255] >= 5000 :
            previousFrame = frame.copy()
            newThresh = thresh # important
            erosion = cv2.erode(newThresh,kernelErosion,iterations =1)
            newThresh = cv2.dilate(newThresh, kernelDilation, iterations=4)
            newThresh = cv2.threshold(newThresh, 250, 255, cv2.THRESH_BINARY)[1]
    

        if newThresh is None:
                newThresh = thresh
        
        #find the contours from the remaining threshold image 
        _, cnts, _ = cv2.findContours(newThresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        #search for any movement
        frame = searchForMovement(cnts, frame)    
        
        #show images
        cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 800,600)
        cv2.imshow("frame", frame)
        cv2.namedWindow('newThresh',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('newThresh', 800,600)
        cv2.imshow("newThresh", newThresh)
        
        
        return frame, previousFrame, thresh, frameDiff


if __name__ == '__main__':


    camera =cv2.VideoCapture(0)  # this for mobil connection cam

    time.sleep(0.25)
    #global hists
    #hists = queue.Queue(10)
    #global prevSum
    #prevSum = 0
    global previousFrame
    #previousFrame = None
    while camera.isOpened():
        
        sensitivity_value = 20
        ret, frame = camera.read()

        #Check if the next camera read is not null
        if ret:
            frame, previousFrame, thresh, frameDiff = trackMotion(frame,previousFrame, sensitivity_value)


        else:
            print("Video Finished")
            close = False
            while not close:
                key1 = cv2.waitKey(3) & 0xFF
                if key1 == 27 or key1 == ord('q'):
                    close = True

            break
        key = cv2.waitKey(3) & 0xFF
        if key == 27 or key == ord('q'):
            print("Bye")
            break

    camera.release()
    cv2.destroyAllWindows()

261181.5
261181.5
261181.5
19105.0
19105.0
19105.0
14453.5
14453.5
14453.5
28448.5
28448.5
28448.5
28448.5
28448.5
28448.5
14298.5
14298.5
14298.5
26871.0
26871.0
26871.0
23476.0
23476.0
30727.5
30727.5
26471.0
26471.0
26471.0
26471.0
26471.0
34424.5
34424.5
34424.5
34424.5
23644.0
23644.0
23644.0
23644.0
23644.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
43324.0
16891.0
16891.0
16891.0
16891.0
16891.0
16891.0
16891.0
16891.0
23506.0
23506.0
47029.5
21769.0
32438.5
29289.0
39547.0
42373.0
94311.5
97847.5
119617.5
116230.5
117465.0
111118.5
115445.0
107972.0
47037.5
47897.0
88611.0
91537.5
38436.0
97247.0
99753.0
92576.5
83785.0
52170.5
52170.5
57354.0
63225.0
44310.0
49789.0
35135.0
74477.5
88676.5
101054.5
104164.0
54612.0
64038.5
64038.5
70206.5
63316.5
41021.5
75900.5
75066.0
43608.0
18777.5
37388.0
23070.5
23070.5
68631.0
64081.0
68836.5
68247.5
61564.5
30206.0
30206.0
35239.0
54433.5
60354.5
60157.0
61268.5
61384

In [78]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)
 
    cv2.imshow('fgmask',frame)
    cv2.imshow('frame',fgmask)

    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    

cap.release()
cv2.destroyAllWindows()